In [1]:
import sail.core
import json
import getpass

#password = None
#password = getpass.getpass(prompt="Password: ")
password = "SailPassword@123"

sail.core.login("lbart@igr.com", password, 6200, "40.71.25.110")

sail.core.load_safe_objects("./SafeObjects")
dg_test_object_guid = "479F9891BBDE41C2B8A06CF2E29F32AC"
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"
dg_sum_column_guid = "441F628312304189886D5074D603C618"

job_id = sail.core.run_job(dg_test_object_guid)
print("Job ID: {} ".format(job_id))

datasets = sail.core.get_datasets()
provision_ds = ""
if ( datasets is not None ):
    json_datasets = json.loads(datasets)
    provision_ds = list(json_datasets.keys())[0]
    print("Datset: {}".format(provision_ds))
else:
    print("No loaded datasets")
    
digital_contracts = sail.core.get_digital_contracts()
provision_dc = ""
if ( digital_contracts is not None):
    json_digital_contracts = json.loads(digital_contracts)
    provision_dc = list(json_digital_contracts.keys())[0]
    print("Digital Contract: {}".format(provision_dc))
else:
    print("No digital contracts")

Job ID: {49C36C59-761B-43D3-BE71-5F85FBF09B3F} 
Datset: {219AA7E2-6253-5549-BBA8-89DF5D77639D}
Digital Contract: {15FA536F-B8A6-4327-A9CF-E63C5C391F87}


In [2]:
job_status = sail.core.get_job_status(job_id)
print("{}:{}".format(job_id,job_status))
#job_status = sail.core.get_job_status(second_job_id)
#print("{}:{}".format(second_job_id,job_status))

{49C36C59-761B-43D3-BE71-5F85FBF09B3F}:Missing Input Parameters


In [3]:
first_provision = sail.core.provision_secure_computational_node(provision_dc, provision_ds, "Standard_D8s_v4")
#second_provision = sail.core.provision_secure_computational_node(provision_dc, provision_ds, "Standard_D8s_v4")

print("Provision ID: {}".format(first_provision))
#print(second_provision)

Provision ID: {"SCNGuid":"{40D76E8C-0A1C-4780-A6A1-B60A17B8FF29}","Status":true}


In [4]:
print(sail.core.wait_for_all_secure_nodes_to_be_provisioned(45000))

{"AllDone":true,"Failed":{},"InProgress":{},"Succeeded":{"{40D76E8C-0A1C-4780-A6A1-B60A17B8FF29}":""}}


In [5]:
my_table_guid = "2534C8FA-0E1A-D442-95A6-8ECB11A39746"
set_ds_response = sail.core.set_parameter(job_id, "4DC7BEC15B384FBD9BD721B7BF36A984", my_table_guid)
#set_ds_response = sail.core.set_parameter(second_job_id, "4C8041B4513A44FAA5409662E014519C", provision_ds)

print("First job IP {}".format(sail.core.get_ip_for_job(job_id)))
#print("Second job IP {}".format(sail.core.get_ip_for_job(second_job_id)))

First job IP 13.92.32.195


In [6]:
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")

pull_data_result = sail.core.pull_data(data_id)
print(pull_data_result)

Success


In [8]:
############################################################################
#
# Produce data frame from CSV
#
############################################################################

import base64
import pickle

message_response = sail.core.wait_for_data(10000)
data_frame = None;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Dataframe has been processed from CSV")
            data_frame = un_serialized_value
            
    message_response = sail.core.wait_for_data(500)


Job 49C36C59761B43D3BE715F85FBF09B3F is done
Dataframe has been processed from CSV


In [9]:
############################################################################
#
# Get the sum of a column
#
############################################################################
dg_sum_column_guid = "441F628312304189886D5074D603C618"
sum_column_job_id = sail.core.run_job(dg_sum_column_guid)

column_name = "BMI"
# Setup a job to injest a dataframe, and sum a column's name
pushed_column_name = sail.core.push_user_data(column_name)
set_ds_response = sail.core.set_parameter(sum_column_job_id, "4E62242A61724E3EBC6C534981D9EE74", my_table_guid)
set_ds_response = sail.core.set_parameter(sum_column_job_id, "4C47725A36DB45A3920714A7089DDDED", data_id)
set_ds_response = sail.core.set_parameter(sum_column_job_id, "4E63DE4EA2A94C53BD658F7DA83A8419", pushed_column_name)

column_result_id = "{}.{}".format(sum_column_job_id, "51559F0423514F6CB38E5E4F8AE3B1EA")

pull_data_result = sail.core.pull_data(column_result_id)

# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("The sum of column {} is {}".format(column_name, un_serialized_value))
            
    message_response = sail.core.wait_for_data(1000)

The sum of column BMI is 7417


In [ ]:
sail.core.deprovision_digital_contract(provision_dc)

In [ ]:
sail.core.exit_current_session()

In [ ]:
safe_functions = sail.core.get_safe_functions()
if ( safe_functions is not None):
    json_safe_functions = json.loads(safe_functions)
    print(json.dumps(json_safe_functions,indent=4))
else:
    print("No safe functions")

In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
tables = sail.core.get_tables()
if ( tables is not None):
    json_tables = json.loads(tables)
    print(json.dumps(json_tables, indent=4))
else:
    print("No tables")


In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"
my_table_guid = "2534C8FA-0E1A-D442-95A6-8ECB11A39746"

increment_job_id = sail.core.run_job(dg_increment_safefunction_guid)
print(increment_job_id)

pull_data_id = "{}.{}".format(increment_job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

pull_data_result = sail.core.pull_data(pull_data_id)
print(pull_data_result)

my_int_guid = sail.core.push_user_data(545)
set_response = sail.core.set_parameter(increment_job_id, "4C21EDF165AB4EF68853FA05FB805A03", my_int_guid)
set_response = sail.core.set_parameter(increment_job_id, "4C8041B4513A44FAA5409662E014519C", my_table_guid)
print(my_int_guid)

print("First job IP {}".format(sail.core.get_ip_for_job(increment_job_id)))

# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(1000)

In [ ]:
second_increment_job_id = sail.core.run_job(dg_increment_safefunction_guid)

# We want the second job's value to be the result of the first job
second_pull_data_id = "{}.{}".format(second_increment_job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

second_pull_data_result = sail.core.pull_data(second_pull_data_id)

set_response = sail.core.set_parameter(second_increment_job_id, "4C21EDF165AB4EF68853FA05FB805A03", pull_data_id)
set_response = sail.core.set_parameter(second_increment_job_id, "4C8041B4513A44FAA5409662E014519C", my_table_guid)

print("Second job IP {}".format(sail.core.get_ip_for_job(second_increment_job_id)))

# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(1000)

In [ ]:
# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(500)

In [ ]:
pull_data_id = "{}.{}".format(job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

pull_data_result = sail.core.pull_data(pull_data_id)
print(pull_data_result)

In [ ]:
digital_contracts = sail.core.get_digital_contracts()
provision_dc = ""
if ( digital_contracts is not None):
    json_digital_contracts = json.loads(digital_contracts)
    provision_dc = list(json_digital_contracts.keys())[0]
    print(provision_dc)
    print(json.dumps(json_digital_contracts, indent=4))
else:
    print("No digital contracts")

In [ ]:
datasets = sail.core.get_datasets()
provision_ds = ""
if ( datasets is not None ):
    json_datasets = json.loads(datasets)
    provision_ds = list(json_datasets.keys())[0]
    print(provision_ds)
    print(json.dumps(json_datasets, indent=4))
else:
    print("No loaded datasets")